In [65]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
import rasterio

In [66]:
# Аутентификация в Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [67]:
df = pd.read_csv("modis_2023_Russian_Federation.csv")

In [71]:
def get_sentinel_image(lat, lon, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(5000).bounds()
    
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
    return image.clip(region)

def get_pre_post_images(fire_lat, fire_lon, fire_date):
    fire_date_ee = ee.Date(fire_date)
    pre_fire_image = get_sentinel_image(fire_lat, fire_lon, fire_date_ee.advance(-1, 'month'), fire_date_ee)
    post_fire_image = get_sentinel_image(fire_lat, fire_lon, fire_date_ee, fire_date_ee.advance(1, 'month'))
    return pre_fire_image, post_fire_image

In [72]:
def get_image_download_url(image, region, scale):
    """
    Получает URL для скачивания изображения из GEE.
    """
    url = image.getDownloadURL({
        'scale': scale,
        'region': region,
        'format': 'GeoTIFF'
    })
    return url

def download_image(url, file_name):
    """
    Загружает изображение по URL и сохраняет его локально.
    """
    import requests
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Изображение сохранено как {file_name}")
    else:
        print(f"Ошибка загрузки: {response.status_code}")

def save_image_locally(image, region, file_name, scale=30):
    """
    Сохраняет изображение из GEE локально.
    """
    url = get_image_download_url(image, region, scale)
    print(url)
    download_image(url, file_name)

def plot_images(pre_image_path, post_image_path):
    """
    Отображает снимки до и после пожара.
    """
    with rasterio.open(pre_image_path) as pre_img:
        pre_image = pre_img.read(1)
    
    with rasterio.open(post_image_path) as post_img:
        post_image = post_img.read(1)
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    ax[0].imshow(pre_image, cmap='gray')
    ax[0].set_title('До пожара')

    ax[1].imshow(post_image, cmap='gray')
    ax[1].set_title('После пожара')

    plt.show()
    
def create_fire_mask(pre_fire_image, post_fire_image):
    pre_nbr = pre_fire_image.normalizedDifference(['B8', 'B12'])
    post_nbr = post_fire_image.normalizedDifference(['B8', 'B12'])
    delta_nbr = pre_nbr.subtract(post_nbr)
    fire_mask = delta_nbr.gt(0.1)
    return fire_mask

In [129]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm  # Для отображения прогресс-бара

ML = 0.00025     # коэффициент масштабирования радиометрии
AL = 0.05        # коэффициент смещения радиометрии
K1 = 765.0       # калибровочная константа K1
K2 = 1275.0      # калибровочная константа K2
epsilon = 0.93   # излучательная способность
lambda_ir = 10.9 * 10**-6  # длина волны ИК канала (мкм)
rho = 1.438 * 10**-2
# Определяем функцию для вычисления маски на основе границ
def create_mask(green_layer, lst_celsius_normalized, green_border, lst_border):
    condition = (green_layer > green_border) & (lst_celsius_normalized > lst_border)
    return np.where(condition, 1, 0)

# Определяем функцию для вычисления коэффициента Метьюса
def calculate_mcc(true_mask, predicted_mask):
    return matthews_corrcoef(true_mask.flatten(), predicted_mask.flatten())

def find_best_borders(tiff_files, step=0.1, delta=0.05):
    best_mcc = -1
    best_green_border = None
    best_lst_border = None

    # Перебор значений green_border и lst_border
    for green_border in tqdm(np.arange(0, 1 + step, step), desc="Перебор значений green_border"):
        for lst_border in tqdm(np.arange(0, 1 + step, step), desc="Перебор значений lst_border", leave=False):
            total_mcc = 0
            count = 0

            for tiff_file in tiff_files:
                with rasterio.open(tiff_file) as dataset:
                    # Считываем слои
                    infrared_layer = dataset.read(4)
                    green_layer = dataset.read(2)
                    true_fire_mask = dataset.read(5)
                    
                    # Преобразуем DN в радиометрическое излучение
                    radiance = ML * infrared_layer + AL
                    brightness_temp = K2 / (np.log((K1 / radiance) + 1))
                    lst = brightness_temp / (1 + (lambda_ir * brightness_temp / rho) * np.log(epsilon))
                    lst_celsius = lst - 273.15
                    lst_celsius_normalized = (lst_celsius - lst_celsius.min()) / (lst_celsius.max() - lst_celsius.min())
                    
                    # Создаем маску на основе границ
                    final_mask = create_mask(green_layer, lst_celsius_normalized, green_border, lst_border)
                    
                    # Вычисляем коэффициент Метьюса
                    mcc = calculate_mcc(true_fire_mask, final_mask)
                    total_mcc += mcc
                    count += 1

            # Средний коэффициент Метьюса
            average_mcc = total_mcc / count

            # Проверяем, является ли текущий результат лучшим
            if average_mcc > best_mcc:
                best_mcc = average_mcc
                best_green_border = green_border
                best_lst_border = lst_border
                print(f"Улучшение результата: Green Border = {best_green_border}, LST Border = {best_lst_border}, MCC = {best_mcc}")

    # Перебор значений с ±delta от найденных лучших границ
    for green_border in [best_green_border + delta, best_green_border, best_green_border - delta]:
        for lst_border in [best_lst_border + delta, best_lst_border, best_lst_border - delta]:
            total_mcc = 0
            count = 0
            
            for tiff_file in tiff_files:
                with rasterio.open(tiff_file) as dataset:
                    # Считываем слои
                    infrared_layer = dataset.read(4)
                    green_layer = dataset.read(2)
                    true_fire_mask = dataset.read(5)
                    
                    # Преобразуем DN в радиометрическое излучение
                    radiance = ML * infrared_layer + AL
                    brightness_temp = K2 / (np.log((K1 / radiance) + 1))
                    lst = brightness_temp / (1 + (lambda_ir * brightness_temp / rho) * np.log(epsilon))
                    lst_celsius = lst - 273.15
                    lst_celsius_normalized = (lst_celsius - lst_celsius.min()) / (lst_celsius.max() - lst_celsius.min())
                    
                    # Создаем маску на основе границ
                    final_mask = create_mask(green_layer, lst_celsius_normalized, green_border, lst_border)
                    
                    # Вычисляем коэффициент Метьюса
                    mcc = calculate_mcc(true_fire_mask, final_mask)
                    total_mcc += mcc
                    count += 1

            # Средний коэффициент Метьюса
            average_mcc = total_mcc / count

            # Проверяем, является ли текущий результат лучшим
            if average_mcc > best_mcc:
                best_mcc = average_mcc
                best_green_border = green_border
                best_lst_border = lst_border
                print(f"Улучшение результата после уточнения: Green Border = {best_green_border}, LST Border = {best_lst_border}, MCC = {best_mcc}")

    return best_green_border, best_lst_border, best_mcc

# Список TIFF файлов
tiff_files = [f"merged/{i:02d}.tiff" for i in range(21)]

# Поиск лучших границ
best_green_border, best_lst_border, best_mcc = find_best_borders(tiff_files)

print(f"Best Green Border: {best_green_border}")
print(f"Best LST Border: {best_lst_border}")
print(f"Best Matthews Correlation Coefficient: {best_mcc}")


Перебор значений lst_border:   5%|▍         | 1/21 [00:02<00:48,  2.42s/it]

Улучшение результата: Green Border = 0.0, LST Border = 0.0, MCC = -0.012598909353627582



Перебор значений lst_border:  10%|▉         | 2/21 [00:03<00:35,  1.86s/it]

Улучшение результата: Green Border = 0.0, LST Border = 0.05, MCC = -0.0073414082301715



Перебор значений lst_border:  38%|███▊      | 8/21 [00:13<00:20,  1.57s/it]

...

Улучшение результата: Green Border = 0.0, LST Border = 0.5, MCC = 0.034160633216983025



Перебор значений lst_border:  57%|█████▋    | 12/21 [00:19<00:13,  1.50s/it]

Улучшение результата: Green Border = 0.0, LST Border = 0.55, MCC = 0.03543340953765905



Перебор значений green_border:  43%|████▎     | 9/21 [06:50<09:07, 45.62s/it][A


KeyboardInterrupt: 